In [1]:
import one_shot_svd
import pdb

In [2]:
pdb.set_trace()
# h, weights = one_shot_svd.generate_model("/home/dshteinbok/nV3_PSFs", "/home/dshteinbok/nV3_PSFs_meta/PSF_Shifts.csv", 
#                                          (800,1280), (640,1024))
# h, weights = one_shot_svd.generate_model("/home/dshteinbok/nV3_PSFs_large", "/home/dshteinbok/nV3_PSFs_meta/PSF_Shifts.csv", 
#                                           (800,1280), (640,1024))
h, weights = one_shot_svd.generate_model("/home/dshteinbok/nV3_PSFs_flat", "/home/dshteinbok/nV3_PSFs_flat_meta/metafile.csv", 
                                          (800,1280), (640,1024))
# one_shot_svd.generate_and_save("/home/dshteinbok/nV3_PSFs_flat", "/home/dshteinbok/nV3_PSFs_flat_meta/metafile.csv", 
#                                           (800,1280), (640,1024), "../data/nV3_flat_h.mat", "../data/nV3_flat_weights.mat")
# one_shot_svd.save_generated(h, weights, "../data/nV3_flat_h.mat", "../data/nV3_flat_weights.mat")

--Return--
None
> /tmp/ipykernel_125746/1185775709.py(1)<cell line: 1>()
----> 1 pdb.set_trace()
      2 # h, weights = one_shot_svd.generate_model("/home/dshteinbok/nV3_PSFs", "/home/dshteinbok/nV3_PSFs_meta/PSF_Shifts.csv",
      3 #                                          (800,1280), (640,1024))
      4 # h, weights = one_shot_svd.generate_model("/home/dshteinbok/nV3_PSFs_large", "/home/dshteinbok/nV3_PSFs_meta/PSF_Shifts.csv",
      5 #                                           (800,1280), (640,1024))

ipdb> c
creating matrix

done

starting svd...

check values of ymat
interpolating...

done




In [29]:
import numpy as np
bad_psf = np.sum(weights[0,0,:]*h, -1)

In [12]:
%time
rectified_bad_psf_np = (bad_psf >= 0.001)*bad_psf

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 15.7 µs


In [37]:
# total number of pixels
P = 1280*800
P

1024000

In [14]:
# total number of nonzero values in mastermat
NNZ = 57*P
NNZ

58368000

In [11]:
bad_psf.dtype

dtype('float64')

In [15]:
# total number of information in mastermat, if it is stored as CSR matrix, in bytes, 
# assuming COL_INDEX and ROW_INDEX take 8 bytes each per value
8*(2*NNZ + P + 1)

942080008

In [41]:
raveled_psf = np.ravel(bad_psf)

In [42]:
%time
np.arange(P)[raveled_psf >= 0.001]

CPU times: user 3 µs, sys: 6 µs, total: 9 µs
Wall time: 17.9 µs


array([526732, 528013, 529291, 529292, 529296, 530567, 530570, 530576,
       531850, 531851, 531853, 531854, 531855, 531856, 533127, 533129,
       533132, 533134, 533136, 533138, 534408, 534410, 534411, 534413,
       534415, 534416, 535691, 535692, 535694, 535696, 535697, 535698,
       536971, 536973, 536976, 536978, 538249, 538250, 538251, 538252,
       538253, 538254, 538255, 538256, 538257, 538258, 538259, 539534,
       539536, 540809, 540812, 540814, 540815, 540816, 540818, 543374,
       543376])

In [51]:
# just for fun, is it feasible to have a list of a hundred million entries in python?
megalist = []
for i in range(100000000):
    megalist.append(i)

In [52]:
import sys
sys.getsizeof(megalist)

835128600

In [ ]:
weights.shape

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

# plt.imshow(np.sum(weights[0,0,:]*h, -1)[350:450,600:700])
# plt.imshow(np.sum(weights[0,0,:]*h, -1))

In [ ]:
import sys
sys.path.append("../tensorflow")
import simulator

flatsim = simulator.ImageSimulator("../data/nV3_flat_h.mat", "../data/nV3_flat_weights.mat")
_, sim = flatsim.simulate("../data/hydra3.jpg", my_simulation=False)
plt.imshow(sim)

In [ ]:
# examine each of the PSFs

# load the metaman for meaningful plot titles (the supposed locations)
import load_PSFs
metaman = load_PSFs.MetaMan("/home/dshteinbok/nV3_PSFs_flat_meta/metafile.csv")
fig, axs = plt.subplots(5,5, figsize=(15,15))
for row in range(5):
    for column in range(5):
        index = row*5 + column
        axs[row,column].imshow(np.sum(weights[metaman.shifts[index+1][1],metaman.shifts[index+1][0],:]*h, -1)[350:450,600:700])
        axs[row, column].set_title(metaman.shifts[index+1])

In [ ]:
img, sim = flatsim.simulate('/home/dshteinbok/TrainingData/Ground_truth_downsampled/1.png', my_simulation=False)
fig, axs = plt.subplots(1, 2, figsize=(15,15))
axs[0].imshow(img)
axs[1].imshow(sim)